In [0]:
#Without Header

spark.sql("SELECT * FROM csv.`/Volumes/workspace/default/practice_dataset_da/BUS424-Netflix_Viewership.csv`").display()

#Without Header using text
spark.sql("SELECT * FROM text.`/Volumes/workspace/default/practice_dataset_da/BUS424-Netflix_Viewership.csv` ").display()

#with Header
df = spark.read.csv("/Volumes/workspace/default/practice_dataset_da/BUS424-Netflix_Viewership.csv",header = True,inferSchema = True)
display(df)

In [0]:
%sql
CREATE TEMPORARY VIEW netflix
USING csv
OPTIONS (
  path "/Volumes/workspace/default/practice_dataset_da/BUS424-Netflix_Viewership.csv",
  header "true",
  inferSchema "true"
);

SELECT * FROM netflix;

In [0]:
%sql
DROP TABLE IF EXISTS workspace.default.netflix_viewrship;


In [0]:
df = spark.read.csv(
    "/Volumes/workspace/default/practice_dataset_da/netflix_viewership_dataset.csv",
    header=True,
    inferSchema=True
)
new_cols = []
for c in df.columns:
    c=c.strip().lower().replace(' ','_')
    new_cols.append(c)
df = df.toDF(*new_cols)
df.write.mode("overwrite").saveAsTable("workspace.default.netflix_viewrship_bronze")


In [0]:
%sql
DESCRIBE EXTENDED workspace.default.netflix_viewrship

In [0]:
%sql
DESCRIBE HISTORY workspace.default.netflix_viewrship

In [0]:
%sql
-- 1. Insert two new movies into the table
INSERT INTO workspace.default.netflix_viewrship
VALUES
(
 'Inception', 'Movie', 'Sci-Fi', 2010, 148,
 'NA', 'NA', 4, 8, 8.8,
 'Available', 'No', 'English', '12,00,00,000'
),
(
 'Avatar', 'Movie', 'Fantasy', 2009, 162,
 'NA', 'NA', 3, 9, 7.8,
 'Available', 'No', 'English', '20,00,00,000'
);

UPDATE workspace.default.netflix_viewrship
SET imdb_rating = 9.0
WHERE title = 'Inception';

DELETE FROM workspace.default.netflix_viewrship
WHERE title = 'Avatar';


In [0]:
%sql
describe  history workspace.default.netflix_viewrship

In [0]:
%sql
select *
from workspace.default.netflix_viewrship version as of 2;


In [0]:
%sql
UPDATE workspace.default.netflix_viewrship
SET title = trim(regexp_replace(title, '[^A-Za-z0-9 ]', ''));
UPDATE workspace.default.netflix_viewrship
SET title = trim(regexp_replace(title, '(?i)\\bseason\\b.*', ''));
UPDATE workspace.default.netflix_viewrship
SET title = trim(regexp_replace(title, '\\b(19|20)[0-9]{2}$', ''));
UPDATE workspace.default.netflix_viewrship
SET viewership = regexp_replace(viewership, '[^0-9]', '');


In [0]:
%sql
CREATE TABLE workspace.default.netflix_viewership_gold AS
SELECT 
    -- Cleaned title
    trim(
        regexp_replace(
            regexp_replace(
                regexp_replace(
                    title, 
                    '[^A-Za-z0-9 ]', ''                 -- remove special chars
                ),
                '(?i)\\bseason\\b.*', ''               -- remove 'season ...'
            ),
            '\\b(19|20)[0-9]{2}$', ''                  -- remove trailing year
        )
    ) AS title,

    -- Cleaned viewership (remove non-numeric)
    regexp_replace(viewership, '[^0-9]', '') AS viewership,

    -- Keep all remaining columns as they are
    *
EXCEPT (title, viewership)

FROM workspace.default.netflix_viewrship_bronze;
